In [ ]:
import ModelingToolkit as Model
import SymPy as sp
import Symbolics as Symb
using DomainSets
import ApproxFun as AF
using NonlinearSolve
import DifferentialEquations as DE
using Random
include("multiharmonic_balance.jl");

Equation parameters

In [ ]:
g = 9.81;
height = 3.0;
omega = 60.0;

Discretization parameters

In [ ]:
harmonics = 1; # number of harmonics
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
Nx = 40;
stepx = (xright-xleft)/Nx;
u0 = rand((Nx+1) * harmonics * 4);

**Declare the problem**

In [ ]:
# Define symbolics
Model.@parameters x, t;

const Dx = Model.Differential(x);
const Dt = Model.Differential(t);

In [ ]:
vars, var_exprs, fields = create_ansatz((x,), t, omega, harmonics, 2);
u = fields[1];
eta = fields[2];
pdes = [Dt(eta) + height * Dx(u),
       Dt(u) + g * Dx(eta)];

In [ ]:
all_sym_eqs = Expr[]

for pde in pdes
    expanded = expand_trig_jl(pde, t, omega)
    eqns = make_equations(expanded, harmonics, omega, t)
    println(eqns)
    sym_eqs = map(transform_sym(Nx) ∘ Meta.parse ∘ string, eqns)
    append!(all_sym_eqs, sym_eqs)
end

resid = create_residual_function(all_sym_eqs, vars, Nx, 0, Dict(
:A1 => ["neumann", :(0)],
:B1 => ["neumann", :(0)],
:C1 => [:(3), nothing],
:D1 => [:(0), "neumann"],)
);

residual! = eval(resid)
N = (Nx+1)

f = NonlinearFunction(residual!)
prob = NonlinearProblem(f, u0, [stepx])

println("Starting now to solve the problem")

sol = solve(prob, RobustMultiNewton(), reltol=1e-5, abstol=1e-5, maxiters=1000)

print(sol.retcode)

In [ ]:
N = Nx + 1

solutions = [reshape(sol.u[(k-1)*N+1:k*N], Nx+1) for k in 1:(2*2*harmonics)];

In [ ]:
using Plots

xgrid = collect(range(start=0.0, stop=1.0, step=stepx))

anim = @animate for t in 1:0.01:10
    if t % 100 == 0
        println(t)
    end
    u_new = zeros(length(xgrid))
        j = 1
        for i in 3:(4*harmonics)
            if isodd(i)
                u_new .+= solutions[i] .* sin(j * omega * t)
            else
                u_new .+= solutions[i] .* cos(j * omega * t)
                j += 1
            end
        end
    plot(xgrid, u_new)
    title!("Shallow Water Equation with H = $(harmonics)", titlefontsize=10)
    ylims!(-5, 5)
end

gif(anim, "SWE_1D_HB.gif", fps=60)